# 📊 Analyse des accidents de la circulation aux États-Unis (2016–2023)

Ce notebook fait partie du pipeline **R&D Résilience Logistique**.  
Il se concentre sur le **module risque routier**, en explorant et nettoyant le dataset **US Accidents** pour fournir des indicateurs exploitables dans le tableau de bord final.


## 📁 Structure du dataset

| Variable             | Description courte                                                 | Type    | Exemples                    |
| -------------------- | ------------------------------------------------------------------ | ------- | --------------------------- |
| `ID`                 | Identifiant unique de l'accident                                   | object  | 'A-1', 'A-2'                |
| `Source`             | Source de la donnée (ex: capteur, site partenaire)                 | object  | 'Source2'                   |
| `Severity`           | Gravité de l'accident (1 à 4)                                      | int     | 2, 3                        |
| `Start_Time`         | Date et heure de début de l'accident                               | datetime| '2016-02-08 05:46:00'       |
| `End_Time`           | Date et heure de fin de l'accident                                 | datetime| '2016-02-08 11:00:00'       |
| `Start_Lat`          | Latitude de début                                                  | float   | 39.865147                   |
| `Start_Lng`          | Longitude de début                                                 | float   | -84.058723                  |
| `End_Lat`            | Latitude de fin (souvent manquante)                                | float   | -                           |
| `End_Lng`            | Longitude de fin (souvent manquante)                               | float   | -                           |
| `Distance(mi)`       | Distance couverte par l'incident                                   | float   | 0.01                        |
| `Description`        | Description textuelle de l'événement                               | object  | 'Accident sur I-70 E...'    |
| `Street`, `City`, `County`, `State`, `Zipcode`, `Country` | Localisation administrative                    | object  | 'Dayton', 'Montgomery', 'OH'|
| `Timezone`           | Fuseau horaire local                                               | object  | 'US/Eastern'                |
| `Airport_Code`       | Code de l'aéroport le plus proche                                  | object  | 'KFFO'                      |
| `Weather_Timestamp`  | Timestamp météo associé                                            | datetime| '2016-02-08 05:58:00'       |
| `Temperature(F)`     | Température en Fahrenheit                                          | float   | 36.9                        |
| `Wind_Chill(F)`      | Ressenti en Fahrenheit                                             | float   | 33.3                        |
| `Humidity(%)`        | Taux d'humidité                                                    | float   | 91.0                        |
| `Pressure(in)`       | Pression atmosphérique en pouces                                   | float   | 29.68                       |
| `Visibility(mi)`     | Visibilité en miles                                                | float   | 10.0                        |
| `Wind_Direction`     | Direction du vent                                                  | object  | 'Calm', 'SW'                |
| `Wind_Speed(mph)`    | Vitesse du vent en mph                                             | float   | 3.5                         |
| `Precipitation(in)`  | Précipitations en pouces                                           | float   | 0.02                        |
| `Weather_Condition`  | Conditions météo décrites                                          | object  | 'Light Rain', 'Overcast'    |
| `Amenity` à `Turning_Loop` | Divers indicateurs routiers booléens (bump, stop, signal, etc.) | bool    | True / False                |
| `Sunrise_Sunset` à `Astronomical_Twilight` | Phase de la journée                          | object  | 'Day', 'Night'              |


# Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from meteostat import Stations, Daily, Hourly

# Réglages pandas & seaborn
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

print("Librairies chargées")

# Chargement des données + premières vérifications

In [ ]:
# Définir le chemin du fichier
file_path = "../data/extracted/USA_Accidents_Traffic/US_Accidents_March23.csv"

# Chargement du CSV
df = pd.read_csv(file_path)

# Vérifier dimensions et aperçu
print(f"Dimensions de df : {df.shape}")
df.head(3)

# Vérification détaillée

In [ ]:
# Affiche infos générales : types, nulls
df.info()

# Statistiques descriptives pour colonnes numériques
df.describe().T

# Liste complète des colonnes pour préparer le tableau résumé
print("\nListe des colonnes :")
print(df.columns.tolist())

# Vérification des doublons et valeurs manquantes 

In [ ]:
# Vérifier le nombre de doublons basés sur l'ID
nb_duplicated = df.duplicated(subset='ID').sum()
print(f"Nombre de doublons basés sur 'ID' : {nb_duplicated}")

# Vérifier le pourcentage de valeurs manquantes par colonne
missing_ratio = df.isnull().mean().sort_values(ascending=False) * 100
print("Pourcentage de valeurs manquantes par colonne (top 10) :")
print(missing_ratio.head(10))

# Conversion des colonnes temporelles + création de Duration(min)
## Objectifs :
- Convertir Start_Time et End_Time en datetime
- Créer la durée en minutes (Duration(min))
- Filtrer les durées aberrantes (exemple : négatives ou trop extrêmes si nécessaire)

In [ ]:
# Convertir et calculer pour inspecter
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')
df['End_Time'] = pd.to_datetime(df['End_Time'], errors='coerce')

# Création de la durée en minutes
df['Duration(min)'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds() / 60

# Vérifier les stats de durée pour décider ensuite quoi faire
print(df['Duration(min)'].describe())

# Vérifier combien de lignes ont une durée négative ou extrême (> 2 jours)
print("\nNombre de lignes avec durée négative :", (df['Duration(min)'] < 0).sum())
print("Nombre de lignes avec durée > 2 jours :", (df['Duration(min)'] > 2880).sum())

In [ ]:
# Vérifier combien de lignes ont Start_Time ou End_Time manquant
missing_start = df['Start_Time'].isnull().sum()
missing_end = df['End_Time'].isnull().sum()

print(f"Lignes sans Start_Time : {missing_start}")
print(f"Lignes sans End_Time : {missing_end}")

# Supprimer ces lignes car sans datetime = pas de durée => inutile pour tableau de bord
initial_shape = df.shape
df = df.dropna(subset=['Start_Time', 'End_Time'])
print(f"Lignes supprimées car Start/End manquant : {initial_shape[0] - df.shape[0]}")

# Filtrer les durées extrêmes (> 2 jours) si nécessaire
print(f"Lignes avant filtrage des durées extrêmes : {df.shape[0]}")
df = df[df['Duration(min)'] <= 2880]
print(f"Lignes après filtrage des durées extrêmes : {df.shape[0]}")

# Distribution de la gravité (Severity)

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Severity', order=sorted(df['Severity'].unique()))
plt.title('Répartition des niveaux de gravité (Severity)')
plt.xlabel('Gravité')
plt.ylabel('Nombre d\'accidents')
plt.show()

# Répartition en pourcentage pour contrôle rapide
severity_counts = df['Severity'].value_counts(normalize=True) * 100
print("Proportion de chaque gravité (%):")
print(severity_counts.sort_index())

In [ ]:
# Distribution de la durée des accidents (log-scale pour lisibilité)

plt.figure(figsize=(12, 6))
sns.histplot(df['Duration(min)'], bins=100)
plt.yscale('log')
plt.title("Distribution des durées d'accidents (minutes)")
plt.xlabel("Durée (min)")
plt.ylabel("Nombre d'incidents (log scale)")
plt.show()

# Afficher les principaux percentiles pour une synthèse rapide
duration_percentiles = df['Duration(min)'].quantile([0.5, 0.75, 0.9, 0.95, 0.99])
print("Percentiles clés des durées (min):")
print(duration_percentiles)

In [ ]:
# Créer la colonne HourOfDay
df['HourOfDay'] = df['Start_Time'].dt.hour

# Afficher la répartition numérique
hour_counts = df['HourOfDay'].value_counts().sort_index()
print("Répartition du nombre d'accidents par heure :")
print(hour_counts)

# Visualiser
plt.figure(figsize=(12,6))
sns.countplot(data=df, x='HourOfDay', color='steelblue')
plt.title("Nombre d'accidents par heure de la journée")
plt.xlabel("Heure (0-23)")
plt.ylabel("Nombre d'incidents")
plt.show()

In [ ]:
# Créer la colonne DayOfWeek
df['DayOfWeek'] = df['Start_Time'].dt.dayofweek  # 0 = lundi, 6 = dimanche

# Afficher la répartition numérique
dow_counts = df['DayOfWeek'].value_counts().sort_index()
print("Répartition du nombre d'accidents par jour de la semaine (0=Lundi) :")
print(dow_counts)

# Visualiser
plt.figure(figsize=(10,5))
sns.countplot(data=df, x='DayOfWeek', color='coral')
plt.title("Nombre d'accidents par jour de la semaine")
plt.xlabel("Jour de la semaine (0 = Lundi)")
plt.ylabel("Nombre d'incidents")
plt.show()


In [ ]:
# Ajouter Month et Season 

# Créer la colonne Month
df['Month'] = df['Start_Time'].dt.month

# Créer la colonne Season selon des règles simples (hémisphère nord)
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['Season'] = df['Month'].apply(get_season)

# Vérifier les répartitions numériques 

print("\nRépartition du nombre d'accidents par mois :")
print(df['Month'].value_counts().sort_index())

print("\nRépartition du nombre d'accidents par saison :")
print(df['Season'].value_counts())

# Visualiser 

plt.figure(figsize=(12,5))
sns.countplot(data=df, x='Month', color='skyblue')
plt.title("Nombre d'accidents par mois")
plt.xlabel("Mois")
plt.ylabel("Nombre d'incidents")
plt.show()

plt.figure(figsize=(8,5))
sns.countplot(data=df, x='Season', order=['Winter', 'Spring', 'Summer', 'Fall'], palette='Set2')
plt.title("Nombre d'accidents par saison")
plt.xlabel("Saison")
plt.ylabel("Nombre d'incidents")
plt.show()


In [ ]:
# Sélectionner les colonnes numériques pertinentes
num_cols = [
    'Severity', 'Duration(min)', 
    'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 
    'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 
    'Precipitation(in)'
]

# Vérifier les colonnes présentes
print(f"Colonnes analysées : {num_cols}")

# Calculer la matrice de corrélation (Spearman plus robuste pour données non linéaires)
corr_matrix = df[num_cols].corr(method='spearman')

# Afficher la heatmap
plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Corrélation Spearman entre facteurs et gravité (Severity)")
plt.show()

# Extraire et afficher le top 5 des facteurs corrélés à la gravité (hors self-corrélation)
severity_corr = corr_matrix['Severity'].drop('Severity').abs().sort_values(ascending=False)
print("\nTop 5 des variables les plus corrélées à la gravité :")
print(severity_corr.head(5))


In [ ]:
# Barplot du top 5 à partir de severity_corr
top_corr = severity_corr.head(5)

plt.figure(figsize=(8,5))
sns.barplot(x=top_corr.values, y=top_corr.index, palette="Blues_d")
plt.title("Top 5 des facteurs corrélés à la gravité (|corr|)")
plt.xlabel("Coefficient de corrélation absolu (Spearman)")
plt.ylabel("Variable")
plt.show()

# Afficher les valeurs exactes pour contrôle
print("Top 5 des facteurs corrélés à la gravité :")
print(top_corr)

In [ ]:
# Normalisation Severity sur [0,1] (1-4 --> 0-1)
df['Severity_norm'] = (df['Severity'] - 1) / 3

# Normalisation Duration sur [0,1] avec seuil max à 2880 min (2 jours)
df['Duration_norm'] = df['Duration(min)'] / 2880
df['Duration_norm'] = df['Duration_norm'].clip(0, 1)

# Marquer une condition météo risquée (simple) : pluie, neige, brouillard, orage
risky_conditions = ['Rain', 'Snow', 'Thunderstorm', 'Fog', 'Heavy Rain', 'Heavy Snow', 'Blowing Snow']
df['Weather_risk'] = df['Weather_Condition'].apply(
    lambda x: any([kw.lower() in str(x).lower() for kw in risky_conditions])
).astype(int)

# Pondération : parts égales
df['Risk_Score'] = (df['Severity_norm'] + df['Duration_norm'] + df['Weather_risk']) / 3

# Contrôle : aperçu
print(df[['Severity', 'Duration(min)', 'Weather_Condition', 'Severity_norm', 'Duration_norm', 'Weather_risk', 'Risk_Score']].head())
print("\nStatistiques du Risk_Score :")
print(df['Risk_Score'].describe())


In [ ]:
# Regrouper par mois
df['Month'] = df['Start_Time'].dt.month

resilience_month = df.groupby('Month')['Risk_Score'].mean().reset_index()
resilience_month['Resilience_Index'] = 1 - resilience_month['Risk_Score']

print(resilience_month)

# Visualiser
plt.figure(figsize=(10,5))
sns.lineplot(data=resilience_month, x='Month', y='Resilience_Index', marker='o')
plt.title("Indice de Résilience Logistique par Mois (1 - Risk_Score moyen)")
plt.xlabel("Mois")
plt.ylabel("Resilience Index")
plt.ylim(0.79, 0.84)  
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,5))

color = 'tab:blue'
ax1.set_xlabel('Mois')
ax1.set_ylabel('Risk_Score moyen', color=color)
ax1.plot(resilience_month['Month'], resilience_month['Risk_Score'], color=color, marker='o')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Resilience Index', color=color)
ax2.plot(resilience_month['Month'], resilience_month['Resilience_Index'], color=color, marker='s')
ax2.tick_params(axis='y', labelcolor=color)

plt.title("Risk_Score & Resilience Index par Mois")
plt.show()


In [ ]:
# Heatmap Risk_Score : Mois vs Jour de la semaine

# Recalculer si nécessaire
df['Month'] = df['Start_Time'].dt.month
df['DayOfWeek'] = df['Start_Time'].dt.dayofweek  # 0 = Lundi

# Grouper
heatmap_data = df.groupby(['Month', 'DayOfWeek'])['Risk_Score'].mean().reset_index()

# Pivot pour format matrice
heatmap_matrix = heatmap_data.pivot(index='DayOfWeek', columns='Month', values='Risk_Score')

print("\nRisk_Score moyen par Mois et Jour de la semaine :")
print(heatmap_matrix.round(4))

# Afficher la heatmap
plt.figure(figsize=(12,6))
sns.heatmap(heatmap_matrix, annot=True, fmt=".3f", cmap="YlOrRd")
plt.title("Heatmap Risk_Score moyen (Mois vs Jour de la semaine)")
plt.xlabel("Mois")
plt.ylabel("Jour de la semaine (0 = Lundi)")
plt.show()

In [ ]:
# Ajouter une colonne Resilience_Index = 1 - Risk_Score
df['Resilience_Index'] = 1 - df['Risk_Score']

# Grouper pour la heatmap
heatmap_resilience = df.groupby(['Month', 'DayOfWeek'])['Resilience_Index'].mean().reset_index()
heatmap_resilience_matrix = heatmap_resilience.pivot(index='DayOfWeek', columns='Month', values='Resilience_Index')

print("\nResilience Index moyen par Mois et Jour de la semaine :")
print(heatmap_resilience_matrix.round(4))

# Afficher
plt.figure(figsize=(12,6))
sns.heatmap(heatmap_resilience_matrix, annot=True, fmt=".3f", cmap="YlGnBu")
plt.title("Heatmap Resilience Index moyen (Mois vs Jour de la semaine)")
plt.xlabel("Mois")
plt.ylabel("Jour de la semaine (0 = Lundi)")
plt.show()

In [ ]:
# Créer la colonne 'HourOfDay' à partir de Start_Time
df['HourOfDay'] = df['Start_Time'].dt.hour

# Calculer Risk_Score moyen et Resilience_Index par heure
risk_hour = df.groupby('HourOfDay')['Risk_Score'].mean().reset_index()
risk_hour['Resilience_Index'] = 1 - risk_hour['Risk_Score']

print("\nRisk_Score et Resilience_Index par heure :")
print(risk_hour)

# Afficher : ligne bleue = Risk_Score, ligne rouge = Resilience_Index
fig, ax1 = plt.subplots(figsize=(12,5))

color = 'tab:blue'
ax1.set_xlabel('Heure de la journée')
ax1.set_ylabel('Risk_Score moyen', color=color)
ax1.plot(risk_hour['HourOfDay'], risk_hour['Risk_Score'], marker='o', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_xticks(range(0,24))

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Resilience_Index', color=color)
ax2.plot(risk_hour['HourOfDay'], risk_hour['Resilience_Index'], marker='s', color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.title("Risk_Score & Resilience_Index par heure de la journée")
plt.show()

In [ ]:
# Définir des catégories de risque logistique basées sur la gravité, durée, météo et heure

def assign_risk_category(row):
    if (row['Severity'] >= 3) and (row['Duration(min)'] > 120):
        return 'High Infrastructure Block'
    elif row['Weather_Condition'] in ['Heavy Rain', 'Snow', 'Thunderstorm', 'Fog']:
        return 'Weather Disruption'
    elif row['HourOfDay'] in range(7,10) or row['HourOfDay'] in range(16,20):
        return 'Peak Hour Congestion'
    else:
        return 'Low Impact'

# S'assurer qu'on a bien la colonne HourOfDay
df['HourOfDay'] = df['Start_Time'].dt.hour

# Appliquer la fonction
df['Risk_Category'] = df.apply(assign_risk_category, axis=1)

In [ ]:
# Calcul final propre
risk_summary = df['Risk_Category'].value_counts().reset_index()
risk_summary.columns = ['Risk_Category', 'Count']
risk_summary['Proportion (%)'] = (risk_summary['Count'] / df.shape[0] * 100).round(2)

print("\nRépartition finale des catégories de risque logistique")
print(risk_summary)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(
    data=risk_summary, 
    y="Risk_Category", 
    x="Count", 
    hue="Risk_Category",  # Fix pour éviter le FutureWarning
    dodge=False,          # pour éviter le doublement
    palette="Set2",
    legend=False
)
plt.title("Répartition finale des types de risque logistique (catégories)")
plt.xlabel("Nombre d'incidents")
plt.ylabel("Catégorie de risque")
plt.show()

In [ ]:
# Moyenne par mois
summary_month = df.groupby('Month')['Risk_Score'].agg(['mean']).reset_index()
summary_month.rename(columns={'mean': 'Risk_Score_mean'}, inplace=True)
summary_month['Resilience_Index_mean'] = 1 - summary_month['Risk_Score_mean']

# Moyenne par jour de la semaine
summary_dayofweek = df.groupby('DayOfWeek')['Risk_Score'].agg(['mean']).reset_index()
summary_dayofweek.rename(columns={'mean': 'Risk_Score_mean'}, inplace=True)
summary_dayofweek['Resilience_Index_mean'] = 1 - summary_dayofweek['Risk_Score_mean']

# Moyenne par heure de la journée
summary_hourofday = df.groupby('HourOfDay')['Risk_Score'].agg(['mean']).reset_index()
summary_hourofday.rename(columns={'mean': 'Risk_Score_mean'}, inplace=True)
summary_hourofday['Resilience_Index_mean'] = 1 - summary_hourofday['Risk_Score_mean']

# Moyenne par catégorie de risque
summary_category = df.groupby('Risk_Category')['Risk_Score'].agg(['mean', 'count']).reset_index()
summary_category.rename(columns={'mean': 'Risk_Score_mean', 'count': 'Count'}, inplace=True)
summary_category['Resilience_Index_mean'] = 1 - summary_category['Risk_Score_mean']
summary_category['Proportion'] = summary_category['Count'] / summary_category['Count'].sum()

# Afficher un échantillon pour vérif rapide
print("\nMoyenne par mois ===")
print(summary_month)

print("\nMoyenne par jour de la semaine ===")
print(summary_dayofweek)

print("\nMoyenne par heure de la journée ===")
print(summary_hourofday)

print("\nMoyenne par catégorie de risque ===")
print(summary_category)


In [ ]:
# Pour graphe par mois
risk_by_month = df.groupby('Month')['Risk_Score'].mean()
resilience_by_month = 1 - risk_by_month

# Idem pour jour de semaine et heure
risk_by_day = df.groupby('DayOfWeek')['Risk_Score'].mean()
resilience_by_day = 1 - risk_by_day

risk_by_hour = df.groupby('HourOfDay')['Risk_Score'].mean()
resilience_by_hour = 1 - risk_by_hour

risk_by_cat = df.groupby('Risk_Category')['Risk_Score'].mean()
resilience_by_cat = 1 - risk_by_cat

In [ ]:
sns.set(style="whitegrid", font_scale=1.2)

# Définir palette
risk_color = "#e74c3c"
resilience_color = "#27ae60"

# Risque & Résilience par Mois ---
plt.figure(figsize=(12,6))
plt.plot(df_month['Month'], df_month['Risk_Score_mean'], '-o', label='Risque', color=risk_color, linewidth=2.5)
plt.plot(df_month['Month'], df_month['Resilience_Index_mean'], '-o', label='Résilience', color=resilience_color, linewidth=2.5)

plt.title("Indice de Risque et de Résilience par Mois")
plt.xlabel("Mois")
plt.ylabel("Score (0-1)")
plt.xticks(df_month['Month'])
plt.ylim(0, 1)
plt.legend()
plt.show()

# Risque & Résilience par Jour de la semaine ---
plt.figure(figsize=(12,6))
plt.plot(df_day['DayOfWeek'], df_day['Risk_Score_mean'], '-o', label='Risque', color=risk_color, linewidth=2.5)
plt.plot(df_day['DayOfWeek'], df_day['Resilience_Index_mean'], '-o', label='Résilience', color=resilience_color, linewidth=2.5)

plt.title("Indice de Risque et de Résilience par Jour de la Semaine (0 = Lundi)")
plt.xlabel("Jour de la Semaine")
plt.ylabel("Score (0-1)")
plt.xticks(df_day['DayOfWeek'], labels=["Lun", "Mar", "Mer", "Jeu", "Ven", "Sam", "Dim"])
plt.ylim(0, 1)
plt.legend()
plt.show()

# Risque & Résilience par Heure ---
plt.figure(figsize=(12,6))
plt.plot(df_hour['HourOfDay'], df_hour['Risk_Score_mean'], '-o', label='Risque', color=risk_color, linewidth=2.5)
plt.plot(df_hour['HourOfDay'], df_hour['Resilience_Index_mean'], '-o', label='Résilience', color=resilience_color, linewidth=2.5)

plt.title("Indice de Risque et de Résilience par Heure de la Journée")
plt.xlabel("Heure")
plt.ylabel("Score (0-1)")
plt.xticks(df_hour['HourOfDay'])
plt.ylim(0, 1)
plt.legend()
plt.show()

# Risque & Résilience par Catégorie ---
plt.figure(figsize=(12,6))
bar_width = 0.35
x = np.arange(len(df_cat['Risk_Category']))

plt.bar(x - bar_width/2, df_cat['Risk_Score_mean'], width=bar_width, color=risk_color, label='Risque')
plt.bar(x + bar_width/2, df_cat['Resilience_Index_mean'], width=bar_width, color=resilience_color, label='Résilience')

plt.title("Comparaison Risque & Résilience par Catégorie de Risque Logistique")
plt.xlabel("Catégorie")
plt.ylabel("Score (0-1)")
plt.xticks(x, df_cat['Risk_Category'], rotation=20)
plt.ylim(0, 1)
plt.legend()
plt.show()


In [ ]:
# Filtrer Weather Disruption et High Infrastructure Block
weather_df = df[df['Risk_Category'] == 'Weather Disruption'].copy()
infra_block_df = df[df['Risk_Category'] == 'High Infrastructure Block'].copy()

# Afficher nombre de lignes
print(f"Weather Disruption : {weather_df.shape[0]} incidents")
print(f"High Infrastructure Block : {infra_block_df.shape[0]} incidents")

# Afficher un échantillon
print("\nWeather Disruption sample :")
print(weather_df.head(3))

print("\nHigh Infrastructure Block sample :")
print(infra_block_df.head(3))


In [ ]:
# Par mois
fig, ax = plt.subplots(1, 2, figsize=(14,5))
sns.countplot(data=weather_df, x='Month', ax=ax[0])
ax[0].set_title("Weather Disruption - par Mois")
sns.countplot(data=infra_block_df, x='Month', ax=ax[1])
ax[1].set_title("High Infrastructure Block - par Mois")
plt.show()

# Par jour de la semaine
fig, ax = plt.subplots(1, 2, figsize=(14,5))
sns.countplot(data=weather_df, x='DayOfWeek', ax=ax[0])
ax[0].set_title("Weather Disruption - par Jour")
sns.countplot(data=infra_block_df, x='DayOfWeek', ax=ax[1])
ax[1].set_title("High Infrastructure Block - par Jour")
plt.show()

# Par heure
fig, ax = plt.subplots(1, 2, figsize=(14,5))
sns.countplot(data=weather_df, x='HourOfDay', ax=ax[0])
ax[0].set_title("Weather Disruption - par Heure")
sns.countplot(data=infra_block_df, x='HourOfDay', ax=ax[1])
ax[1].set_title("High Infrastructure Block - par Heure")
plt.show()

# === B) Conditions météo ===
print("\nTop 10 Weather Conditions pour Weather Disruption :")
print(weather_df['Weather_Condition'].value_counts().head(10))

# === C) Statistiques Durée et Gravité ===
print("\nStats Weather Disruption:")
print(weather_df[['Duration(min)', 'Severity']].describe())

print("\nStats High Infrastructure Block:")
print(infra_block_df[['Duration(min)', 'Severity']].describe())

In [ ]:
# Conserver uniquement incidents avec Weather Disruption OU Peak Hour Congestion
df_heat = df[df['Risk_Category'].isin(['Peak Hour Congestion', 'Weather Disruption'])].copy()

# Remplacer 'None' par 'Clear/Other' pour contrôle mais on peut filtrer ensuite
df_heat = df_heat[df_heat['Main_Weather'] != 'None']

# Si tu veux limiter aux types météo clés uniquement (Rain, Snow, Fog, Thunderstorm)
df_heat = df_heat[df_heat['Main_Weather'].isin(['Rain', 'Snow', 'Fog', 'Thunderstorm'])]

# Regrouper Risk_Score moyen par heure et météo
heatmap_data = df_heat.groupby(['Main_Weather', 'HourOfDay'])['Risk_Score'].mean().reset_index()

# Pivot pour heatmap
heatmap_matrix = heatmap_data.pivot(index='Main_Weather', columns='HourOfDay', values='Risk_Score')

# Tri de l'index pour l'ordre que tu veux
heatmap_matrix = heatmap_matrix.loc[['Fog', 'Rain', 'Snow', 'Thunderstorm']]

# Afficher
plt.figure(figsize=(16,6))
sns.heatmap(heatmap_matrix, annot=True, fmt=".2f", cmap="YlOrRd")
plt.title("Heatmap Risque moyen (Weather Conditions vs Heure de la journée)")
plt.xlabel("Heure")
plt.ylabel("Condition météo")
plt.show()

In [ ]:
# Étiquette météo simplifiée pour ceux affectés par la météo
conditions = [
    df['Risk_Category'] == 'Weather Disruption',
]
choices = [
    df['Weather_Condition'].where(df['Risk_Category'] == 'Weather Disruption', None)
]

df['Main_Weather'] = np.select(conditions, choices, default=None)

# Nettoyer : regrouper les valeurs proches
df['Main_Weather'] = df['Main_Weather'].replace({
    'Heavy Rain': 'Rain',
    'Light Rain': 'Rain',
    'Snow': 'Snow',
    'Heavy Snow': 'Snow',
    'Thunderstorm': 'Thunderstorm',
    'Fog': 'Fog'
}).fillna('None')

# Vérif : risk & resilience sont déjà calculés ===
df['Resilience_Index'] = 1 - df['Risk_Score']

# Vérif : catégorie risque déjà correcte ===
print(df['Risk_Category'].value_counts())


# Export 
cols_to_export = [
    'ID', 'Start_Time', 'Severity', 'Duration(min)',
    'Risk_Score', 'Resilience_Index', 'Risk_Category',
    'Main_Weather',
    'Month', 'DayOfWeek', 'HourOfDay'
]

output_path = "../data/cleaned/usa_accidents_traffic_cleaned.csv"
df[cols_to_export].to_csv(output_path, index=False)

print(f"Export fait : {output_path}")
print(f"Colonnes exportées : {cols_to_export}")
print(f"Lignes : {df.shape[0]}")

In [ ]:
total = df.shape[0]
peak_hour_count = df[df['Risk_Category'] == 'Peak Hour Congestion'].shape[0]
infra_block_mean = infra_block_df['Duration(min)'].mean()
weather_count = weather_df.shape[0]

peak_hour_pct = peak_hour_count / total * 100
weather_pct = weather_count / total * 100

print(f"""
Synthèse du risque routier :
- Plus de {peak_hour_pct:.1f}% des incidents surviennent lors des heures de pointe, ce qui en fait la principale cause de congestion logistique.
- Les incidents « Infrastructure Bloquée » représentent des blocages longs, avec une durée moyenne de {infra_block_mean:.0f} minutes.
- Les conditions météo sévères sont moins fréquentes ({weather_pct:.1f}% des cas) mais génèrent un risque plus élevé, notamment en cas de neige, brouillard ou orage.
""")


# Résumé final du module Analyse du Risque Routier

## Objectif
Ce notebook a permis de nettoyer, structurer et analyser le dataset US Accidents (2016–2023)
dans le cadre du pipeline R&D pour mesurer la résilience logistique.
L'objectif principal est de fournir un fichier prêt à être intégré dans un tableau de bord.

## Ce qui a été fait
- Chargement et nettoyage des données : conversion des timestamps, calcul de la durée en minutes,
  suppression des valeurs aberrantes (durées négatives ou supérieures à 2 jours).
- Exploration descriptive : distribution des niveaux de gravité, distribution des durées,
  identification des pics horaires, jours et mois à risque.
- Corrélation Spearman pour détecter les variables influentes sur la gravité.
- Création d'un score de risque standardisé (Risk_Score) combinant gravité, durée et météo.
- Calcul d'un indice de résilience (Resilience_Index) comme complément de l'indice de risque.
- Attribution d'une catégorie de risque logistique pour chaque incident :
  - Peak Hour Congestion (heures de pointe)
  - High Infrastructure Block (incidents bloquants longue durée)
  - Weather Disruption (conditions météo critiques)
  - Low Impact (autres cas mineurs)
- Génération de visualisations pour valider la saisonnalité, la cyclicité horaire et les pics par catégorie.
- Export final d'un fichier CSV structuré contenant toutes les colonnes utiles :
  ID, Start_Time, Severity, Duration(min), Risk_Score, Resilience_Index, Risk_Category,
  Main_Weather (regroupement des conditions météo dominantes), Month, DayOfWeek, HourOfDay.

## Analyse synthétique
- La majorité des incidents sont classés en Low Impact, avec un risque faible pour la logistique.
- Les congestions horaires (matin et fin de journée) représentent environ 42% des incidents.
- Les incidents bloquants (High Infrastructure Block) sont rares (environ 3%) mais leur durée
  moyenne est élevée (~300 min), ce qui impacte fortement la fluidité des réseaux.
- Les perturbations liées à la météo (Weather Disruption) sont limitées (environ 2% des cas) mais
  concentrées sur des événements de forte pluie, neige ou brouillard.

## Conclusion
Ce module fournit une base cohérente pour alimenter un tableau de bord opérationnel permettant
de suivre en temps réel ou en historique les risques routiers pour la chaîne logistique.
Les indicateurs produits sont prêts pour une visualisation interactive et un croisement éventuel
avec d'autres modes de transport.